# Pytorch Lighting training
> on huggingface transformers

In [2]:
# default_exp hf.train

In [16]:
# export
from forgebox.hf.data import IOBES
from forgebox.imports import *
from forgebox.loop import chunkify
import pytorch_lightning as pl
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    pipeline
)
from tqdm.notebook import tqdm
from typing import Callable, List
from torch import device

In [290]:
# export
try:
    ishell = get_ipython()
    IS_JUPYTER = True
    from tqdm.notebook import tqdm
except NameError:
    IS_JUPYTER = False
    from tqdm import tqdm

In [6]:
# !pip install transformers==4.9.1
# !pip install pytorch-lightning==1.3.8
# !pip install tensorflow==2.2.0

## Load model and tokenizer

In [17]:
# export

# ner model and tokenizer
def ner_model_from(
    name:str, dataset: IOBES
):
    """
    name: from_pretrain(name)
    """
    model = AutoModelForTokenClassification.from_pretrained(
        name,
        num_labels=len(dataset.cates),
    )
    dataset.set_hfconfig(model.config)
    return model

def ner_tokenizer_from(
    name: str
):
    return AutoTokenizer.from_pretrained(
        name, add_prefix_space=True)

## Lightning data module

In [8]:
# export

# ner data module
class NERDataModule(pl.LightningDataModule):
    def __init__(self, train_ds, val_ds, batch_size=32):
        super().__init__()
        self.train_ds = train_ds
        self.val_ds = val_ds
        self.batch_size = batch_size

    def train_dataloader(self):
        return self.train_ds.dataloader(batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return self.val_ds.dataloader(batch_size=self.batch_size*2, shuffle=False)

In [10]:
# export

# ner module
class NERModule(pl.LightningModule):
    """
    PyTorch lightning module for training ner model
    """
    def __init__(
        self, model,
        ):
        """
        model: huggingface transformer model for ner
        """
        super().__init__()
        self.model = model

    def forward(self, batch):
        return self.model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            labels=batch['labels'])
    
    def training_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        self.log("loss", loss)
        self.log("acc", self.calcualte_acc(outputs, batch.labels))
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(batch)
        loss = outputs.loss
        self.log("val_loss", loss)
        self.log("val_acc", self.calcualte_acc(outputs, batch.labels))
        return loss
    
    def calcualte_acc(self, outputs, labels):
        pred_idx = outputs.logits.argmax(-1)
        mask = torch.ones_like(pred_idx)
        mask[labels==-100]=False
        return (pred_idx[mask]==labels[mask]).float().mean()
    
    def configure_optimizers(self):
        # discriminative learning rate
        param_groups = [
            {'params': self.model.roberta.parameters(), 'lr': 5e-6},
            {'params': self.model.classifier.parameters(), 'lr': 1e-3},
        ]
        optimizer = torch.optim.Adam(param_groups, lr=1e-3)
        return optimizer

## Enhance pipeline

In [35]:
# export
def clean_ner_output(self, outputs):
    """
    Cleaning output for NER task
    """
    results = []
    current = []
    last_idx = 0
    # make to sub group by position
    for output in outputs:
        if output["start"] in [last_idx, last_idx-1]:
            current.append(output)
        else:
            results.append(current)
            current = [output, ]
        last_idx = output["end"]
    if len(current) > 0:
        results.append(current)

    # from tokens to string
    strings = []
    for c in results:
        tokens = []
        starts = []
        ends = []
        for o in c:
            tokens.append(o['word'])
            starts.append(o['start'])
            ends.append(o['end'])

        new_str = self.tokenizer.convert_tokens_to_string(tokens)
        if new_str != '':
            strings.append(dict(
                word=new_str,
                start=min(starts),
                end=max(ends),
                entity=c[0]['entity_group']
            ))
    return strings

In [281]:
# export
class NERInference:
    """
    NER Inference pipeline
    ner = NERInference.from_pretrained('xxxx/xxxx')
    ner.predict(['text1','text2'])
    """

    def __init__(self, model, tokenizer, name=None):
        super().__init__()
        self.model = model.eval()
        self.tokenizer = tokenizer
        self.name = name if name else "NER model"

    def __repr__(self):
        return f"[NERInference on {self.name}]"

    def to(self, device_str):
        self.model = self.model.to(device(device_str))
        return self

    @classmethod
    def from_pretrained(cls, tag):
        """
        Load from pretrained model and tokenizer
        """
        model = AutoModelForTokenClassification.from_pretrained(tag)
        tokenizer = AutoTokenizer.from_pretrained(tag)
        return cls(model=model, tokenizer=tokenizer, name=model.config._name_or_path)
    
    def __call__(self, data, batch_size=32, dev=device("cpu")):
        if type(data) == str:
            return self.batch_predict([data,])
        else:
            return self.predict(data, dev=dev, batch_size=batch_size)

    def predict(
        self,
        texts: List[str],
        dev=device("cpu"),
        batch_size: int = 32,
        progress_bar: bool = True
    ) -> pd.DataFrame:
        """
        Predict a list of sentences/ paragraphs
        """
        # place the model into device
        self.model = self.model.to(dev)
        iterator = list(enumerate(chunkify(texts, bs=batch_size)))
        if progress_bar:
            iterator = tqdm(iterator, leave=False)

        # run through iterator
        all_dfs = []
        for i, text_b in iterator:
            # by batch prediction
            batch_df = self.batch_predict(text_b)
            if len(batch_df) > 0:
                # calculate the row number
                batch_df['text_id'] = batch_df.apply(
                    lambda row: i*batch_size+row.batch_row_sn, axis=1)
                all_dfs.append(batch_df)

        # place the model back to cpu
        self.model = self.model.to("cpu")
        return pd.concat(all_dfs).reset_index(drop=True)
    
    def tokenizing(self, texts):
        inputs = self.tokenizer(
            texts,
            padding="max_length",
            max_length=self.tokenizer.model_max_length,
            return_attention_mask=True,
            return_tensors='pt', truncation=True, return_offsets_mapping=True
        ).to(self.model.device)
        return inputs


    def batch_predict(self, texts:List[str])-> pd.DataFrame:
        """
        Predict a single batch of sentences
        """
        id2label = self.model.config.id2label
        inputs = self.tokenizing(texts)

        with torch.no_grad():
            outputs = self.model(input_ids=inputs.input_ids,
                                 attention_mask=inputs.attention_mask)
        inputs = inputs.to(device('cpu'))

        pred_idx = outputs.logits.argmax(-1).to(device("cpu"))
        batch_size = pred_idx.size(0)
        offsets = inputs.offset_mapping
        results = []
        for bi in range(batch_size):
            text = texts[bi]
            input_ids = inputs.input_ids[bi]
            word_ids = inputs.word_ids(bi)
            pred_ids = pred_idx[bi]
            # initial  values for the row
            last_pos = 0
            previous_has_positive = False
            current_start = 0
            current_index = 0
            current_id = 0
            line = []
            for ti in range(1, len(input_ids)):
                if input_ids[ti] == self.tokenizer.sep_token_id:
                    break
                # is the current token an appending sub-word?
                if word_ids[ti] == last_pos:
                    pass
                # is current token negative
                elif pred_ids[ti].item() == 0:
                    # store the previous hanging prediction
                    if previous_has_positive:
                        start = current_start
                        end = offsets[bi, ti, 0].item()
                        line.append({
                            "start": start, "end": end,
                            "entity": id2label[current_id],
                            "word": text[start:end],
                            "index": current_index,
                        })

                    current_start = offsets[bi, ti, 0].item()
                    previous_has_positive = False
                    current_id = 0
                    current_index = ti
                # has positive prediction index, other than zero
                else:
                    if previous_has_positive:
                        # different than the previous
                        if current_id != pred_ids[ti].item():
                            start = current_start
                            end = offsets[bi, ti, 0].item()
                            line.append({
                                        "start": start,
                                        "end": end,
                                        "entity": id2label[current_id],
                                        "word": text[start:end],
                                        "index": current_index,
                                        })
                            current_start = offsets[bi, ti, 0].item()
                    # this is the 1st postive predict for a while
                    else:
                        current_start = offsets[bi, ti, 0].item()
                    previous_has_positive = True
                    current_index = ti
                    current_id = pred_ids[ti].item()

                last_pos = word_ids[ti]
            if previous_has_positive:
                start = current_start
                end = offsets[bi, ti, 1].item()
                line.append({
                            "start": start,
                            "end": end,
                            "entity": id2label[current_id],
                            "word": text[start:end],
                            "index": current_index,
                            })

            results.append(line)
        all_dfs = []
        for i, res in enumerate(results):
            sub_df = pd.DataFrame(res)
            sub_df["batch_row_sn"] = i
            all_dfs.append(sub_df)
        return pd.concat(all_dfs)